In [1]:
from hack_itau_quant import EfficientFrontier
import yfinance as yf
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import pandas as pd

In [67]:
tickers = ['BPAC11.SA', 'ITUB4.SA', 'OIBR3.SA', 'PETR4.SA']

close_prices = yf.download(tickers, start='2017-01-01', end='2021-01-01')['Close']

returns = close_prices.pct_change()[1:]

expected_returns = returns.mean()
cov_matrix = returns.cov()

[*********************100%***********************]  4 of 4 completed


In [68]:
ef = EfficientFrontier(expected_returns, cov_matrix)

In [149]:
returns, risks = [], []
weights = []

for r in tqdm(np.arange(0.0009, 0.003, 0.00005)):

    w = ef.efficient_return(r).reshape(4, 1)
    
    w = w / np.sum(w)
    
    if np.sum(w) != 1:
        print(w, np.sum(w))
    
    weight_dict = {tickers[i]: float(w[i]) for i in range(len(tickers))}
    weights.append(weight_dict)

    rs = np.dot(w.T, expected_returns)

    sigma = np.sqrt(np.dot(w.T, np.dot(cov_matrix, w)))

    returns.append(float(rs) * 252)
    risks.append(float(sigma) * np.sqrt(252))

100%|██████████| 43/43 [00:00<00:00, 1774.73it/s]

[[0.17665065]
 [0.66517978]
 [0.11584856]
 [0.04232101]] 1.0000000000000002
[[0.23258636]
 [0.5996078 ]
 [0.11726311]
 [0.05054273]] 1.0000000000000002
[[0.37242562]
 [0.43567784]
 [0.1207995 ]
 [0.07109704]] 1.0000000000000002
[[0.42836133]
 [0.37010585]
 [0.12221405]
 [0.07931876]] 1.0000000000000002
[[0.62413631]
 [0.14060391]
 [0.12716499]
 [0.10809479]] 1.0000000000000002
[[ 0.90381484]
 [-0.18725602]
 [ 0.13423777]
 [ 0.14920341]] 0.9999999999999998
[[ 0.95975055]
 [-0.252828  ]
 [ 0.13565232]
 [ 0.15742513]] 0.9999999999999999
[[ 0.9877184 ]
 [-0.28561399]
 [ 0.1363596 ]
 [ 0.16153599]] 1.0000000000000002
[[ 1.26739694]
 [-0.61347392]
 [ 0.14343237]
 [ 0.2026446 ]] 1.0000000000000002


In [144]:
def to_percentage(weight):
    
    return str(weight * 100) + '%'

In [151]:
df = pd.DataFrame(weights)

df = (df * 100).round(2).astype(str) + '%'

df['Returns'] = returns
df['Volatilies'] = risks
df['Sharpe'] = (df['Returns']) / (df['Volatilies'])

In [152]:
df.head()

,BPAC11.SA,ITUB4.SA,OIBR3.SA,PETR4.SA,Returns,Volatilies,Sharpe
0,14.87%,69.8%,11.51%,3.82%,0.2268,0.323636,0.700788
1,17.67%,66.52%,11.58%,4.23%,0.2394,0.325452,0.735592
2,20.46%,63.24%,11.66%,4.64%,0.2520,0.327777,0.768816
3,23.26%,59.96%,11.73%,5.05%,0.2646,0.330599,0.800365
4,26.06%,56.68%,11.8%,5.47%,0.2772,0.333906,0.830173


In [153]:
import plotly.express as px

hover_data = ['Sharpe', *tickers]

max_sharpe = df[df.Sharpe == df.Sharpe.max()]
min_vol = df[df.Volatilies == df.Volatilies.min()]

fig = px.line(df, x="Volatilies", y="Returns", hover_data=hover_data, title="Efficient Frontier")
fig.data[0].update(mode='markers+lines')

fig.add_scatter(x=max_sharpe.Volatilies, y=max_sharpe.Returns)
fig.add_scatter(x=min_vol.Volatilies, y=min_vol.Returns)

fig.show()

In [110]:
min_vol

,BPAC11.SA,ITUB4.SA,OIBR3.SA,PETR4.SA,Returns,Volatilies,Sharpe
0,14.87,69.8,11.51,3.82,0.2268,0.323636,0.700788


In [108]:
import plotly.graph_objects as go


fig = go.Figure()

fig.add_trace(go.Scatter(x=df.Volatilies, y=df.Returns,
                    mode='lines+markers',
                    name='Efficient Frontier',
                    customdata=np.dstack((z2, z3)),
    hovertemplate='<b>z1:%{z:.3f}</b><br>z2:%{customdata[0]:.3f} <br>z3: %{customdata[1]:.3f} '))

# fig.add_trace(go.Scatter(x=df.Volatilies, y=df.Returns,
#                     mode='markers',
#                     name='Portfolio'))

max_sharpe = df[df.Sharpe == df.Sharpe.max()]

fig.add_trace(go.Scatter(x=max_sharpe.Volatilies, y=max_sharpe.Returns,
                    mode='markers',
                    name='Best Sharpe'))

fig.show()

NameError: name 'z2' is not defined

In [43]:
df[df.Sharpe == df.Sharpe.max()]

,BPAC11.SA,ITUB4.SA,OIBR3.SA,PETR4.SA,Returns,Volatilies,Sharpe
26,0.875847,-0.15447,0.13353,0.145093,0.0022,0.031229,1.118333
